In [74]:
from functools import cmp_to_key
from collections import Counter
import math

In [1]:
def load_text(filename):
    with open(f"../Data/{filename}", 'r', encoding='utf-8-sig') as f:
        text = f.read()
        return text

In [2]:
czech = load_text('czech.txt')
english = load_text('english.txt')
french = load_text('french.txt')
german = load_text('german.txt')
hungarian = load_text('hungarian.txt')
unknown0 = load_text('unknown0.txt')
unknown1 = load_text('unknown1.txt')
unknown2 = load_text('unknown2.txt')

texts = [
    ('Czech', czech),
    ('English', english),
    ('French', french),
    ('German', german),
    ('Hungarian', hungarian)
]

In [67]:
def burrows_wheeler(text):
    text2 = text + text
    n = len(text)
    
    indices = sorted(range(0, n), key=cmp_to_key(lambda a, b: -1 if text2[a:a+n] < text2[b:b+n] else 1))
    #print(indices)
    
    first = [text2[i] for i in indices]
    last = [text2[i-1] for i in indices]
    vector = [-1] * n
    
    freq = Counter(first)
    spaces = {}
    f = 0
    for key in sorted(freq.keys()):
        spaces[key] = f
        f = f + freq[key]
        
    for i, item in enumerate(last):
        vector[i] = spaces[item]
        spaces[item] = spaces[item] + 1
    
    #print(f"spaces={spaces}")
    
    #print(f"last={last}")
    #print(f"first={first}")
    #print(f"vector={vector}")
    
    return last, first, vector, first.index(text[0])

text = "ema ma maso"
last, first, vector, first_index = burrows_wheeler(text)
assert last == ['a', 'a', 'm', 'm', 'm', 'o', 'e', ' ', ' ', 's', 'a']
assert first == [' ', ' ', 'a', 'a', 'a', 'e', 'm', 'm', 'm', 'o', 's']
assert vector == [2, 3, 6, 7, 8, 9, 5, 0, 1, 10, 4]
assert first_index == 5

In [104]:
def move_to_front(values):
    alphabet = list(sorted(set(values)))
    result = []
    
    for item in values:
        value = alphabet.index(item)
        result.append(value)
        
        #print(f"value={item}")
        #print(f"before = {alphabet}")
        for i in reversed(range(value)):
            t = alphabet[i]
            alphabet[i] = alphabet[i + 1]
            alphabet[i + 1] = t
        #print(f"after = {alphabet}")
    
    return result

assert move_to_front("abcddcbamnopponm") == [0, 1, 2, 3, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3]

In [68]:
def entropy(text):
    counter = Counter(text)
    n = len(text)
    entropy = 0
    for char in counter:
        pi = counter[char] / n
        entropy += pi * math.log(pi, 2)
    return -entropy

In [106]:
print(f"{'File':<12} {'Before':>10} {'After':>10}")
for name, text in texts:
      last, first, vector, first_index = burrows_wheeler(text)
      encoded = move_to_front(last)
      
      before = entropy(text)
      after = entropy(encoded)
      print(f"{name:<12} {before:>10.3f} {after:>10.3f}")

File             Before      After
Czech             5.068      3.066
English           4.650      2.772
French            4.908      2.722
German            4.726      2.854
Hungarian         4.819      3.208
